<a href="https://colab.research.google.com/github/hincz-lab/motion-blur-microscopy/blob/main/Training_Material/Train_Phase_One/Motion_Blur_Phase_One_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This code will be used to train the phase one segmentation network for use on MBM images and videos. To begin, let's first clone a Github directory to work in.

You will need to go into the next block of code by double clicking the "Show Code" button underneath the "Clone Github Repository" header. Please replace the code <token> with your personal access token. 

To generate a personal access token, in Github, click on your icon at the top right and choose settings, then click on developer settings, and finally, create a personal access token. So, its settings -> Developer Settings -> Personal Access Token (Classic).

After cloning the repository, you can see the repository in the "Files" section of Colab, located on the left-hand side of the screen.

In [ ]:
#@title Clone Github Repository
!git clone https://<token>@github.com/hincz-lab/motion-blur-microscopy.git

Cloning into 'motion-blur-microscopy'...
remote: Enumerating objects: 415, done.
remote: Counting objects: 100% (270/270), done.
remote: Compressing objects: 100% (207/207), done.
remote: Total 415 (delta 119), reused 138 (delta 59), pack-reused 145
Receiving objects: 100% (415/415), 45.23 MiB | 21.54 MiB/s, done.
Resolving deltas: 100% (166/166), done.


Next, we import any libraries or packages we may need for the rest of this document.

In [ ]:
#@title Import Packages And Libraries
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import load_model, Model, model_from_json
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras import activations 
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Conv2DTranspose, Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, BatchNormalization, Activation
from tensorflow.keras.layers import concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import Sequence
from tensorflow.keras import backend as K

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os as os
import shutil
import cv2 as cv2

from random import randint
from random import random

# Setup

We begin by first importing all of the MBM training images and their respective labeled and layered masks. You can import the MBM training images to the directory **motion-blur-microscopy -> Training_Material -> Train_Phase_One -> Original_Tiles**. You can import all of the corresponding labeled and layered masks to the directory **motion-blur-microscopy -> Training_Material -> Train_Phase_One -> Labeled_Layered_Tiles**.

Note, that MBM training images and their respective labeled and layered masks should have the same name in both directories. The MBM training images should be .png files, and the labeled and layered masks should be .npy files. Furthermore, the MBM training images and labeled and layered masks should be the resized version (128x128).

Once you have uploaded your MBM training images and labeled and layered masks, you can run the next block of code, which will complete two tasks.


1.   The code will move all of the uploaded documents to a seperate directory at **motion-blur-microscopy -> Training_Material -> Train_Phase_One -> Flow_Folder**
2.   The code will create a .csv document, which will contain the names of all of the MBM training images and their respective labeled and layered masks. 

Both the flow folder and the .csv document are necessary for the training of the phase one segmentation network.



In [ ]:
#@title Prepare Data For training

base_Directory = "motion-blur-microscopy/Training_Material/Train_Phase_One/"

for subdirectory in os.listdir(base_Directory):
  if '.ipynb_checkpoints' in os.listdir(base_Directory + subdirectory):
    os.rmdir(base_Directory + subdirectory + "/.ipynb_checkpoints")
  if 'blank.txt' in os.listdir(base_Directory + subdirectory):
      os.remove(base_Directory + subdirectory + "/blank.txt")

image_Names = os.listdir(base_Directory + "Original_Tiles/")
mask_Names = os.listdir(base_Directory + "Labeled_Layered_Tiles/")
flow_Directory = base_Directory + "Flow_Folder"
data_Frame_Location = base_Directory + "Excel_Directory/Training_File_Names.csv"

for file in os.listdir(base_Directory + "Original_Tiles/"):
  shutil.copy(base_Directory + "Original_Tiles/" + file, base_Directory + "Flow_Folder/" + file)

for file in os.listdir(base_Directory + "Labeled_Layered_Tiles/"):
  shutil.copy(base_Directory + "Labeled_Layered_Tiles/" + file, base_Directory + "Flow_Folder/" + file)

dict = {'X': np.sort(image_Names), 'Y_True': np.sort(mask_Names)} 
training_Data_Frame = pd.DataFrame(dict)
training_Data_Frame.to_csv(data_Frame_Location)

# Training

To complete the training, we will first establish the architecture of our segmentation network by running the following block of code.

In [ ]:
#@title Create Architecture

# ================= Phase 1 Models ====================

def Phase1_Net(img_size, num_classes):
    inputs = Input(shape=img_size + (3,))

    x = Conv2D(64,kernel_size = 3, strides = (1,1),
                            padding = "same")(inputs)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)
    
    previous_block_concatenate1 = x
    x = MaxPooling2D(pool_size = (2,2),
                                  strides = (2,2))(x)

    x = Conv2D(128,kernel_size = 3, strides = (1,1),
                            padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)
    x = MaxPooling2D(pool_size = (2,2),
                                  strides = (2,2))(x)

    previous_block_concatenate2 = x

    concate_block_num = 3
    for filters in [256, 512, 512]:
        x = Conv2D(filters,3, strides = (1,1),
                            padding = "same")(x)
        x = BatchNormalization()(x)
        x = Activation(activations.relu)(x)
        x = Conv2D(filters,3, strides = 1,
                         padding = "same")(x)
        x = BatchNormalization()(x)
        x = Activation(activations.relu)(x)
        x = MaxPooling2D(pool_size = (2,2),
                                  strides = (2,2))(x)
        globals()['previous_block_concatenate%s' % concate_block_num] = x
        concate_block_num = concate_block_num + 1
        print(("No errors for filter size:" + str(filters)))



    x = Conv2D(512,3, strides = 1,
                            padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)
    x = MaxPooling2D(pool_size = (2,2),
                                  strides = (2,2))(x)

    x = Conv2D(512,3, strides = 1,
                            padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    x = Conv2DTranspose(256,2, strides = (2,2))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    x = concatenate([x, previous_block_concatenate5], axis =-1)

    x = Conv2DTranspose(256,2, strides = (2,2))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    x = concatenate([x, previous_block_concatenate4],axis=-1)

    x = Conv2DTranspose(128,2, strides = (2,2))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    x = concatenate([x, previous_block_concatenate3],axis=-1)
    
    x = Conv2DTranspose(64,2, strides = (2,2))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    x = concatenate([x, previous_block_concatenate2],axis=-1)


    x = Conv2DTranspose(32,2, strides = (2,2))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    x = Conv2DTranspose(64,2, strides = (2,2))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)


    x = concatenate([x, previous_block_concatenate1],axis=-1)

    x = Conv2D(32,3, strides = (1,1),
                            padding = 'same')(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)
    x = Conv2D(num_classes,3, strides = (1,1),
                            padding = 'same')(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)
    outputs = Conv2D(num_classes,3, strides = (1,1),
                            activation = 'softmax',
                            padding = 'same',
                            name = 'sRBC_classes')(x)
    model = Model(inputs,outputs)

    return model

# ================ Train Phase 2 Model ================

# training the model for a specific amount of epochs 
def Phase1_train_network(model, X_train, y_train, 
                        X_test, y_test, epochs):
    
    train_history = model.fit(X_train, y_train, epochs=epochs, 
                              validation_data=(X_test, y_test),
                              shuffle = True, verbose = 2)
    return model, train_history

Next, we create a data generator, which will read MBM training images and their labeled and layered masks in in batches, which will reduce memory requirements.

In [ ]:
#@title Create Data Generator

class DataGenerator(Sequence):
    def __init__(self, data_Frame, x_Col, y_Col, directory,tile_Namesake, mask_Namesake, subset = None,
                 horizontal_Flips = False, vertical_Flips = False, rotations = False, batch_Size = 32,
                 split = False, training_Ratio = 1, shuffle = False, dim = (128,128), number_Of_Channels = 3,
                 number_Of_Classes = 2, sample_Mean_Zero_Center_Standarardization = True, number_Of_Training_Images = None):
        self.batch_Size = batch_Size
        self.df = data_Frame
        self.x_Col = x_Col
        self.y_Col = y_Col
        self.dim = dim
        self.directory = directory
        self.subset = subset
        self.sample_Mean_Zero_Center_Standarardization = sample_Mean_Zero_Center_Standarardization
        self.number_Of_Classes = number_Of_Classes
        self.number_Of_Channels = number_Of_Channels
        self.shuffle = shuffle
        self.tile_Names = self.df[self.x_Col]
        self.truth_Names = self.df[self.y_Col]
        self.tile_Namesake = tile_Namesake
        self.mask_Namesake = mask_Namesake
        self.horizontal_Flips = horizontal_Flips
        self.vertical_Flips = vertical_Flips
        self.number_Of_Training_Images = number_Of_Training_Images
        self.index_List = np.arange(number_Of_Training_Images) + 1
        self.rotations = rotations
        self.training_Samples = int(training_Ratio*len(self.index_List))
        if split == True:
            self.train_Index_List = self.index_List[:self.training_Samples]
            self.validate_Index_List = self.index_List[self.training_Samples:]
        else:
            self.train_Index_List = self.index_List[:]
            self.validate_Index_List = []
        if self.shuffle == True:
            self.on_Epoch_End()
    def __len__(self):
        return int(len(self.train_Index_List)/self.batch_Size)
    def __getitem__(self, index):
        if self.subset == "Training":
            indexes = self.train_Index_List[index*self.batch_Size:(index*self.batch_Size) + self.batch_Size]
            X, y_True = self.generate_Batch(indexes)
        elif self.subset == "Validation":
            indexes = self.validate_Index_List[index*self.batch_Size:(index*self.batch_Size) + self.batch_Size]
            X, y_True = self.generate_Batch(indexes)
        else:
            indexes = self.train_Index_List[index*self.batch_Size:(index*self.batch_Size) + self.batch_Size]
            X, y_True = self.generate_Batch(indexes)
        return X, y_True
    def on_Epoch_End(self):
        if self.shuffle == True:
            np.random.shuffle(self.train_Index_List)
    def generate_Batch(self,indexes):
        X = np.zeros((self.batch_Size, *self.dim, self.number_Of_Channels))
        y_True = np.zeros((self.batch_Size, *self.dim, self.number_Of_Classes))
        for index in range(len(indexes)):
            if self.sample_Mean_Zero_Center_Standarardization == True:
                img = plt.imread(self.directory + self.tile_Namesake + str(indexes[index]) + ".png")[:,:,0:3]
                if np.max(img) == int(np.max(img)) and len(str(np.max(img))) == len(str(int(np.max(img)))):
                    img = img.copy()/255.
                if len(np.shape(img)) == 2:
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                if np.shape(img)[2] == 4:
                    img = img.copy()[:,:,0:3]
                mask = np.load(self.directory + self.mask_Namesake + str(indexes[index]) + ".npy")
                img, mask = self.augment_Image(img,mask)
                X[index,:,:,:] = self.standard_norm(plt.imread(self.directory + self.tile_Namesake + str(indexes[index]) + ".png")[:,:,0:3])
                y_True[index,:,:,:] = np.load(self.directory + self.mask_Namesake + str(indexes[index]) + ".npy")
            else:
                img = plt.imread(self.directory + self.tile_Namesake + str(indexes[index]) + ".png")[:,:,0:3]
                if np.max(img) == int(np.max(img)) and len(str(np.max(img))) == len(str(int(np.max(img)))):
                    img = img.copy()/255.
                if len(np.shape(img)) == 2:
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                if np.shape(img)[2] == 4:
                    img = img.copy()[:,:,0:3]
                mask = np.load(self.directory + self.mask_Namesake + str(indexes[index]) + ".npy")
                img, mask = self.augment_Image(img,mask)
                X[index,:,:,:] = img
                y_True[index,:,:,:] = mask
        return X, y_True
    def standard_norm(self,img):
        height, width, channels = img.shape
        for channel in range(channels):
            img[:,:,channel] = (img[:,:,channel] - np.mean(img[:,:,channel]))/np.std(img[:,:,channel])
        return img
    def augment_Image(self, image, mask):
        if self.rotations == True:
            random_Integer = randint(1,5)
            image = np.rot90(image.copy(),random_Integer)
            mask = np.rot90(mask.copy(),random_Integer)
        
        if self.horizontal_Flips == True:
            random_Float = random()

            if random_Float < 0.5:
                image = np.flip(image.copy(),0)
                mask = np.flip(mask.copy(),0)
        if self.vertical_Flips == True:
            random_Float = random()

            if random_Float < 0.5:
                image = np.flip(image.copy(),1)
                mask = np.flip(mask.copy(),1)
        return image, mask

Finally, we can train our phase one network, using the following block of code. Note, that although we are training for 1000 epochs, that we have a condition where the training stops if no improvement is found in 10 conescutive epochs.

In [ ]:
#@title Train Phase One Segmentation Network

tile_Names_Style = "/" 
mask_Names_Style = "/"
img_size = (128,128)
num_classes = 2
number_Of_Epochs = 1000
model = Phase1_Net(img_size, num_classes)
#learning hyperparamters for the training optimizer 
model.compile(Adam(lr=0.001),
                 metrics = ['accuracy'],
                 loss = tf.keras.losses.CategoricalCrossentropy())
# model.compile(Adam(lr=0.001),
#                 metrics = ['accuracy'],
#                 loss = iou)
train_Gen = DataGenerator(data_Frame=training_Data_Frame,
                    x_Col = "X",
                    y_Col = "Y_True",
                    directory = flow_Directory,
                    vertical_Flips=True,
                    horizontal_Flips = True,
                    rotations = True,
                    split = True,
                    training_Ratio = 0.8,
                    shuffle = True,
                    tile_Namesake = tile_Names_Style,
                    mask_Namesake = mask_Names_Style,
                    subset = "Training",
                    number_Of_Training_Images = len(image_Names))
validate_Gen = DataGenerator(data_Frame=training_Data_Frame,
                    x_Col = "X",
                    y_Col = "Y_True",
                    directory = flow_Directory,
                    vertical_Flips=False,
                    horizontal_Flips = False,
                    rotations = False,
                    split = True,
                    training_Ratio = 0.8,
                    shuffle = True,
                    tile_Namesake = tile_Names_Style,
                    mask_Namesake = mask_Names_Style,
                    subset = "Validation",
                    number_Of_Training_Images = len(image_Names))
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto', restore_best_weights=True)
checkpoint = ModelCheckpoint("Phase_One.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
training_History = model.fit(train_Gen,
                             validation_data = validate_Gen,
                                 epochs=number_Of_Epochs, 
                                 verbose = 1, callbacks = [early, checkpoint])


No errors for filter size:256
No errors for filter size:512
No errors for filter size:512
Epoch 1/1000
12/12 [==============================] - ETA: 0s - loss: 0.3920 - accuracy: 0.9620 
Epoch 1: val_loss improved from inf to 0.14630, saving model to Phase_One.h5
12/12 [==============================] - 275s 22s/step - loss: 0.3920 - accuracy: 0.9620 - val_loss: 0.1463 - val_accuracy: 0.9966
Epoch 2/1000
12/12 [==============================] - ETA: 0s - loss: 0.3061 - accuracy: 0.9877 
Epoch 2: val_loss improved from 0.14630 to 0.12432, saving model to Phase_One.h5
12/12 [==============================] - 240s 20s/step - loss: 0.3061 - accuracy: 0.9877 - val_loss: 0.1243 - val_accuracy: 0.9969
Epoch 3/1000
12/12 [==============================] - ETA: 0s - loss: 0.2492 - accuracy: 0.9883 
Epoch 3: val_loss improved from 0.12432 to 0.09746, saving model to Phase_One.h5
12/12 [==============================] - 239s 20s/step - loss: 0.2492 - accuracy: 0.9883 - val_loss: 0.0975 - val_accu

# Save Model

In [ ]:
#@title Save Model
model.save(base_Directory + "/Phase_One_Model/Phase_One_Trained_Network.h5")

The trained network will be saved to the location **motion-blur-microscopy -> Training_Material -> Train_Phase_One -> Phase_One_Model**.